<a href="https://colab.research.google.com/github/vaibhav720/AI-Conversational-Contest/blob/main/ConverationalContest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install transformers

In [2]:
import tensorflow as tf  # Replace with your chosen framework
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Attention
import tensorflow_datasets as tfds

dataset = tfds.load('huggingface:gigaword', split="train")  # Change "train" to "test" or "validation" as needed



/usr/local/lib/python3.10/dist-packages/tensorflow_datasets/core/dataset_builders/huggingface_dataset_builder.py:160: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  hf_names = hf_datasets.list_datasets()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3803957 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/189651 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1951 [00:00<?, ? examples/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

...

Generating validation examples...:   0%|          | 0/189651 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/gigaword/1.2.0.incompleteENY8QN/gigaword-validation.tfrecord*...:   0%|   …

Generating test examples...:   0%|          | 0/1951 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/gigaword/1.2.0.incompleteENY8QN/gigaword-test.tfrecord*...:   0%|         …

Dataset gigaword downloaded and prepared to /root/tensorflow_datasets/gigaword/1.2.0. Subsequent calls will reuse this data.


In [3]:
# Load your dataset (replace with your specific data loading code)
data_test = tfds.load("huggingface:gigaword", split="test")

In [19]:
data_train=dataset
# Convert TensorFlow dataset to a list of strings
dataset_strings = [example["document"].numpy().decode("utf-8") for example in data_train.take(5000)]
dataset_strings

["australia 's current account deficit shrunk by a record #.## billion dollars -lrb- #.## billion us -rrb- in the june quarter due to soaring commodity prices , figures released monday showed .",
 'at least two people were killed in a suspected bomb attack on a passenger bus in the strife-torn southern philippines on monday , the military said .',
 'australian shares closed down #.# percent monday following a weak lead from the united states and lower commodity prices , dealers said .',
 "south korea 's nuclear envoy kim sook urged north korea monday to restart work to disable its nuclear plants and stop its `` typical '' brinkmanship in negotiations .",
 'south korea on monday announced sweeping tax reforms , including income and corporate tax cuts to boost growth by stimulating sluggish private consumption and business investment .',
 'taiwan share prices closed down #.## percent monday on wall street weakness and lacklustre interim earnings from electronics manufacturing giant hon h

In [15]:
# Load Gigaword dataset
dataset = tfds.load("gigaword", split="train")

In [20]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

# Preprocess function
def preprocess(example):
    text = example["document"]
    inputs = tokenizer(text, return_tensors="tf", max_length=128, truncation=True, padding="max_length")
    labels = tf.identity(inputs["input_ids"])[:, 1:]  # Shift labels to the right
    inputs["labels"] = labels
    return inputs

# Tokenize and preprocess dataset
dataset = dataset_strings.map(preprocess).batch(8)


# Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss={"logits": loss}, metrics=["accuracy"])
model.fit(dataset, epochs=1)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


AttributeError: 'list' object has no attribute 'map'

In [ ]:
def create_model(vocab_size, embedding_dim, units):
  model = Sequential()
  model.add(Embedding(vocab_size, embedding_dim, input_length=MAX_LENGTH))
  model.add(LSTM(units, return_sequences=True))  # Adjust layers as needed
  model.add(Attention())  # Add attention mechanism if desired
  model.add(LSTM(units))
  model.add(Dense(vocab_size, activation="softmax"))
  return model

model = create_model(len(tokenizer.word_index) + 1, 128, 64)  # Adjust hyperparameters
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


Epoch 1/10


OperatorNotAllowedInGraphError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    OperatorNotAllowedInGraphError: Exception encountered when calling layer 'transformer_model_4' (type TransformerModel).
    
    in user code:


In [ ]:
def create_model(vocab_size, embedding_dim, units):
  model = Sequential()
  model.add(Embedding(vocab_size, embedding_dim, input_length=MAX_LENGTH))
  model.add(LSTM(units, return_sequences=True))  # Adjust layers as needed
  model.add(Attention())  # Add attention mechanism if desired
  model.add(LSTM(units))
  model.add(Dense(vocab_size, activation="softmax"))
  return model

model = create_model(len(tokenizer.word_index) + 1, 128, 64)  # Adjust hyperparameters
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
model.fit(sequences_train, ..., epochs=10, validation_data=(sequences_test, ...))


In [ ]:
# Evaluate on test data
loss, accuracy = model.evaluate(sequences_test, ...)
print(f"Test accuracy: {accuracy}")

# Predict on new input
new_sentence = "This is a sentance with grammer errors."
new_sequence = tokenizer.texts_to_sequences([new_sentence])
new_sequence = pad_sequences(new_sequence, maxlen=MAX_LENGTH)
prediction = model.predict(new_sequence)
corrected_sentence = tokenizer.sequences_to_texts(np.argmax(prediction, axis=1))
print(f"Corrected sentence: {corrected_sentence[0]}")
